In [100]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%reload_ext autoreload
%autoreload 2
from HelperFunctions import minibatch, down_sample

In [101]:
df=pd.read_csv('final_df.csv',index_col='id')
mybatch=minibatch(df)

In [102]:
df.head()

,funded_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,profit,RANDOM
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,3600.0,36,13.99,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,2015-12-01,Fully Paid,debt_consolidation,190xx,PA,5.91,0.0,2003-08-01,30.0,-999.0,0.0,29.7,13.0,w,0.0,30.0,Individual,0.0,722.0,36.0,34.0,9300.0,4.0,0.0,0.0,3.0,2.0,0.0,3.0,76.9,0.0,0.0,178050.0,13734.0,677.0,137158.0,0.692308,0.230769,0.076923,0.555556,0.400000,0.444444,0.666667,0.444444,0.538462,37.566667,4421.723917,0.952901
68355089,24700.0,36,11.99,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,2015-12-01,Fully Paid,small_business,577xx,SD,16.06,1.0,1999-12-01,6.0,-999.0,0.0,19.2,38.0,w,0.0,-999.0,Individual,0.0,0.0,73.0,29.0,111800.0,6.0,0.0,0.0,2.0,0.0,0.0,2.0,97.4,0.0,0.0,314017.0,24667.0,717.0,164921.0,0.710526,0.157895,0.105263,0.629630,0.294118,0.185185,0.166667,0.740741,0.578947,6.100000,25679.660000,0.743162
68341763,20000.0,60,10.78,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,2015-12-01,Fully Paid,home_improvement,605xx,IL,10.78,0.0,2000-08-01,-999.0,-999.0,0.0,56.2,18.0,w,0.0,-999.0,Joint App,0.0,0.0,73.0,65.0,14000.0,1.0,0.0,0.0,14.0,0.0,0.0,0.0,100.0,0.0,0.0,218418.0,14877.0,697.0,171003.0,0.388889,0.333333,0.277778,0.571429,0.500000,0.428571,0.166667,0.571429,0.333333,18.266667,22705.924294,0.119479
68476807,10400.0,60,22.45,F,F1,3 years,MORTGAGE,104433.0,Source Verified,2015-12-01,Fully Paid,major_purchase,174xx,PA,25.37,1.0,1998-06-01,12.0,-999.0,0.0,64.5,35.0,w,0.0,-999.0,Individual,0.0,0.0,84.0,78.0,34000.0,3.0,0.0,0.0,4.0,0.0,0.0,4.0,96.6,0.0,0.0,439570.0,88097.0,697.0,235962.0,0.542857,0.285714,0.171429,0.473684,0.444444,0.315789,0.300000,0.368421,0.342857,7.100000,11740.500000,0.731115
68426831,11950.0,36,13.44,C,C3,4 years,RENT,34000.0,Source Verified,2015-12-01,Fully Paid,debt_consolidation,300xx,GA,10.20,0.0,1987-10-01,-999.0,-999.0,0.0,68.4,6.0,w,0.0,-999.0,Individual,0.0,0.0,99.0,76.0,12900.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,100.0,0.0,0.0,16900.0,4000.0,692.0,0.0,0.666667,0.333333,0.000000,0.500000,1.000000,0.750000,0.500000,1.000000,0.833333,17.233333,13708.948530,0.072298


#### Prototype Code: 

In [33]:
month_grouped=df.groupby('issue_d',as_index=False)
month_sampled_df=month_grouped.apply(lambda x: x.sample(frac=0.10,random_state=0))
month_sampled_df.index=month_sampled_df.index.droplevel()
default_rate_df=pd.DataFrame(month_sampled_df.groupby('grade').apply(lambda x:(x['loan_status']=='Default').sum()/x['loan_status'].count()),columns=['Default_Rate'])
# default_rate_df

In [42]:
# First code for specific subgrade
# A_df=(month_sampled_df.groupby('grade').get_group('A'))
# Fully_Paid_A_df=A_df[A_df['loan_status']=='Fully Paid']
# Default_A_df=A_df[A_df['loan_status']=='Default']

In [62]:
np.random.seed(0)
down_sampled_df=pd.DataFrame()
for grade in ['A','B','C','D','E','F','G']:
    print(grade)
    if (default_rate_df.loc[grade,:][0] <=0.5):
        # in the case where we need to down sample fully paid:
        # stratifying by grade
        grade_df=(month_sampled_df.groupby('grade').get_group(grade))
        # Dividing into fully paid and default
        Fully_Paid_grade_df=grade_df[grade_df['loan_status']=='Fully Paid']
        Default_grade_df=grade_df[grade_df['loan_status']=='Default']
        # down sample fully paid sample size into default sample size
        down_sample_size=Default_grade_df.shape[0]
        down_sampled_index=np.random.choice(Fully_Paid_grade_df.index,size=down_sample_size,replace=False)
        down_sampled_Fully_Paid_grade_df=Fully_Paid_grade_df.loc[down_sampled_index,:]
        # Combining the new downsampled dataframes together
        down_sampled_df=pd.concat([down_sampled_df,down_sampled_Fully_Paid_grade_df])
        down_sampled_df=pd.concat([down_sampled_df,Default_grade_df])
    else: 
        # in the case where we need to down sample default: 
        # stratifying by grade
        grade_df=(month_sampled_df.groupby('grade').get_group(grade))
        # Dividing into fully paid and default
        Fully_Paid_grade_df=grade_df[grade_df['loan_status']=='Fully Paid']
        Default_grade_df=grade_df[grade_df['loan_status']=='Default']
        # down sample fully paid sample size into default sample size
        down_sample_size=Default_grade_df.shape[0]
        down_sampled_index=np.random.choice(Default_grade_df.index,size=down_sample_size,replace=False)
        down_sampled_Default_grade_df=Default_grade_df.loc[down_sampled_index,:]
        # Combining the new downsampled dataframes together
        down_sampled_df=pd.concat([down_sampled_df,down_sampled_Default_grade_df])
        down_sampled_df=pd.concat([down_sampled_df,Fully_Paid_grade_df])

A
B
C
D
E
F
G


In [104]:
down_sampled_df=down_sample(df)
print(down_sampled_df.shape)
down_sampled_df.head()

(52858, 57)


,funded_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,profit,RANDOM
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94327724,40000.0,36,6.99,A,A2,2 years,MORTGAGE,160000.0,Verified,2016-12-01,Fully Paid,debt_consolidation,085xx,NJ,5.25,0.0,2002-02-01,-999.0,-999.0,0.0,30.8,26.0,w,0.0,-999.0,Individual,0.0,0.0,80.0,40.0,61500.0,3.0,0.0,0.0,15.0,0.0,0.0,0.0,100.0,0.0,0.0,76500.0,15000.0,742.0,0.0,0.846154,0.153846,0.000000,0.636364,0.428571,0.272727,0.25,0.363636,0.346154,23.333333,43813.339841,0.122137
43976612,7000.0,36,6.92,A,A4,-999,MORTGAGE,33600.0,Verified,2015-04-01,Fully Paid,debt_consolidation,329xx,FL,13.71,0.0,2004-10-01,-999.0,-999.0,0.0,11.7,16.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,-999.0,13500.0,-999.0,0.0,0.0,8.0,0.0,0.0,1.0,100.0,0.0,0.0,25601.0,12101.0,797.0,0.0,0.625000,0.312500,0.062500,0.700000,0.285714,0.200000,-999.00,0.500000,0.375000,24.366667,7614.417064,0.569026
71758277,6250.0,36,5.32,A,A1,10+ years,MORTGAGE,85000.0,Source Verified,2016-02-01,Fully Paid,debt_consolidation,959xx,CA,6.38,0.0,2004-11-01,-999.0,-999.0,0.0,7.5,31.0,w,0.0,-999.0,Individual,0.0,0.0,63.0,22.0,27700.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,93.5,0.0,0.0,225700.0,10000.0,767.0,143202.0,0.774194,0.161290,0.064516,0.583333,0.214286,0.208333,0.20,0.541667,0.483871,26.333333,6654.688130,0.699403
76266206,25000.0,36,5.32,A,A1,8 years,RENT,135000.0,Not Verified,2016-04-01,Fully Paid,debt_consolidation,335xx,FL,20.75,0.0,2001-07-01,-999.0,-999.0,0.0,48.7,19.0,w,0.0,-999.0,Individual,0.0,0.0,34.0,45.0,53300.0,1.0,0.0,0.0,7.0,0.0,0.0,1.0,100.0,0.0,0.0,142384.0,84614.0,712.0,0.0,0.473684,0.421053,0.052632,0.888889,0.625000,0.666667,0.25,1.000000,0.631579,28.400000,26999.279503,0.181346
29704332,24000.0,36,7.69,A,A4,10+ years,MORTGAGE,130000.0,Verified,2014-10-01,Fully Paid,debt_consolidation,070xx,NJ,12.79,0.0,1994-05-01,-999.0,-999.0,0.0,85.5,13.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,-999.0,17100.0,-999.0,0.0,0.0,8.0,0.0,0.0,1.0,100.0,0.0,0.0,392655.0,19200.0,727.0,310431.0,0.538462,0.230769,0.153846,1.000000,0.428571,0.428571,-999.00,0.571429,0.538462,36.533333,26951.209574,0.268982
